# Data Audit Report

This is the first part of our Competition 2 where we performed our preprocessing steps on the data. More details can be found in our [README.MD](README.md) file.

### Importing Our Required Packages and things

In [1]:
#importing required libraries and packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backend as bk
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from pandas import ExcelWriter
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection
from sklearn.model_selection import cross_val_score

#set plot style to 'ggplot' and don't limit the view of DF when diaplyhign them to screen
plt.style.use('ggplot')
pd.options.display.max_columns = None

### Read the Data In

In [2]:
df = pd.read_csv('Data/Output.csv')
df = df.drop(columns = ['Unnamed: 0'])
df['Def'] = df.Default
df = df.drop(columns = ['Default'])
df.head()

,Debt_Apr_May,Debt_May_Jun,Debt_Jun_Jul,Debt_Jul_Aug,Debt_Aug_Sept,Def
0,0.0,0.0,1.0,1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0,1.0,1.0
2,1.0,-1.0,-1.0,-1.0,1.0,0.0
3,-1.0,-1.0,1.0,-1.0,-1.0,0.0
4,1.0,-1.0,1.0,-1.0,1.0,0.0


In [3]:
#Copy our target variables to their own df
df_target = df[['Def']].copy()
df_target.head(10)
#change the data type to categorical
df_target['Def'] = pd.Categorical(df_target.Def)
#Drop Default from our target variable df
df = df.drop(['Def'], axis=1)

In [4]:
#chacking that our data was transfered properly.
df_target.head(5)

,Def
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0


### Changing Our DataTypes

We want to make our fields from integer to float so we do that here

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size= 0.20, random_state=2019)
oversample = pd.concat([X_train,y_train],axis=1)
max_size = oversample['Def'].value_counts().max()
lst = [oversample]
    
for class_index, group in oversample.groupby('Def'):
    lst.append(group.sample(max_size-len(group), replace=True))
X_train = pd.concat(lst)
y_train=pd.DataFrame.copy(X_train['Def'])
del X_train['Def']

In [6]:
xgb_model = xgb.XGBClassifier(seed=2019, min_child_weight=1, max_depth=6, learning_rate= 0.2, n_estimators=150, objective='binary:logistic')
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)

/home/jupyter-bwalsh/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(metrics.accuracy_score(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.80      0.55      0.65      4654
        1.0       0.26      0.54      0.35      1343

avg / total       0.68      0.55      0.58      5997

[[2559 2095]
 [ 624  719]]
0.5466066366516592


In [8]:
feats = xgb_model.feature_importances_ * 100
cols = df.columns
for x in range(0,len(cols)):
    print(cols[x],'------------', feats[x])

Debt_Apr_May ------------ 21.785501
Debt_May_Jun ------------ 21.48592
Debt_Jun_Jul ------------ 18.274416
Debt_Jul_Aug ------------ 18.801678
Debt_Aug_Sept ------------ 19.652485


In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
scores = cross_val_score(xgb_model, X_test, y_test.Def.ravel(), cv=kfold)

In [10]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.03)


## Rerun XGBoost with only top 5 important featrues